### RAG pipeline: Data ingestion to vector db pipeline

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
folder_path = "../data/pdf"
file_path = "pdf_1.pdf"
full_path = os.path.join(folder_path, file_path)
loader = PyPDFLoader(full_path)
documents = loader.load()


text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)
texts = text_splitter.split_text(documents[0].page_content)
texts